<div align="center">
  <img src="Bases/logoMoSEF.jpeg" width="90px" align="left">
  <div align="right">Enseignant : Nexialog</div>
  <div align="right">Réalisé par : Seçil Coskun, ABABII Anisoara, Eunice KOFFI et DIAKITE Gaoussou</div>
  <div align="right">Année : 2022/2023</div>
  <br><br><br>
  <div align="center">
    <span style="font-family:Lucida Caligraphy;font-size:32px;color:darkgreen">Master 2 Modélisation Statistiques Economiques et Financières</span>
  </div>
  <br>
  <div align="center">
    <span style="font-family:Lucida Caligraphy;font-size:32px;color:darkgreen">Université Paris 1 Panthéon-Sorbonne</span>
  </div>
  <br>
  <div align="center">
    <span style="font-family:Lucida Caligraphy;font-size:28px;background-color:#F5DEB3;padding:5px;border-radius:10px">Challenge Nexialog</span>
  </div>
  <br><br>
  <hr>
  <div align="center">
    <h1 style="font-size: 30px">Partie 2 : Modèle</h1>
    <p style="font-size: 24px;color:#696969">Ce notebook présente le modèle BMA utilisé pour prédire le taux de défaut et les différents tests d'hypothèses appliqué.</p>
  </div>
</div>


<div align="left"><span style="font-family:Lucida Caligraphy;font-size:20px;color:darkgreen"> <span style="color:green">      
IMPORTATION DES LIBRAIRIES</span>

In [1]:
import warnings
import pandas as pd
import numpy as np
from sas7bdat import SAS7BDAT
import matplotlib.pyplot as plt
from statsmodels.tsa.vector_ar.vecm import coint_johansen
%matplotlib inline
from arch.unitroot import PhillipsPerron
from scipy.stats import kendalltau
warnings.filterwarnings("ignore")
from statsmodels.stats.diagnostic import acorr_ljungbox
from statsmodels.tsa.seasonal import seasonal_decompose
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from typing import List
from statsmodels.tsa.stattools import adfuller, kpss 
import statsmodels.api as sm
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf
from scipy.stats import kendalltau
from scipy.stats import levene
from statsmodels.tsa.stattools import adfuller
from statsmodels.stats.outliers_influence import variance_inflation_factor
import statsmodels.api as sm
from scipy.stats import shapiro
from statsmodels.stats.diagnostic import het_white
import itertools
import statsmodels.formula.api as smf
from statsmodels.tools.tools import add_constant

<div align="left"><span style="font-family:Lucida Caligraphy;font-size:20px;color:darkgreen"> <span style="color:green">      
IMPORTATION DE LA BASE DE DONNEES POUR LE MODELE</span>

In [2]:
df = pd.read_csv("Bases\data_model.csv", index_col="Date")
df.head(3)

,gr_RGDP,gr_IRLT,gr_UNR,gr_RGDP_lag1,gr_RGDP_lag2,gr_RGDP_lag3,gr_RGDP_lag4,gr_IRLT_lag1,gr_IRLT_lag2,gr_UNR_lag1,gr_UNR_lag2,gr_UNR_lag3,gr_UNR_lag4,gr_HICP_lissee_lag2,gr_HICP_lissee_lag3,gr_HICP_lissee_lag4,DR_lag1,DR
Date,,,,,,,,,,,,,,,,,,
2011-10-31,-0.024919,0.382664,0.276238,-0.009517,0.005452,0.015567,0.022698,0.571053,0.354839,0.108471,0.020272,-0.067907,-0.059051,0.022307,0.021585,0.018661,0.048630,0.043486
2012-01-31,-0.032554,0.173554,0.341834,-0.024919,-0.009517,0.005452,0.015567,0.382664,0.571053,0.276238,0.108471,0.020272,-0.067907,0.026800,0.022307,0.021585,0.043486,0.043242
2012-04-30,-0.032510,0.098901,0.264994,-0.032554,-0.024919,-0.009517,0.005452,0.173554,0.382664,0.341834,0.276238,0.108471,0.020272,0.030172,0.026800,0.022307,0.043242,0.040473


<hr style="border-width:2px;border-color:green">
<center><h1>Modèle BMA</h1></center>
<hr style="border-width:2px;border-color:green">

Le modèle BMA (Bayesian Model Averaging) est une méthode qui permet de prendre en compte plusieurs modèles de régression linéaire et d'obtenir une prédiction en combinant les résultats de ces modèles. Dans ce notre cas, nous avons utilisé la bibliothèque statsmodels pour créer une boucle qui génère toutes les combinaisons possibles de variables explicatives à partir de notre base de données. Pour chaque combinaison, nous avons ajusté un modèle de régression linéaire en utilisant les variables explicatives et la variable cible DR. Ensuite, nous avons stocké chaque modèle dans une liste et avons calculé les prévisions correspondantes. La méthode BMA permet ensuite de pondérer les prévisions de chaque modèle pour obtenir une prédiction globale plus précise.

In [3]:
models = []
y_pred = []

variables = [var for var in df.columns if var != "DR"]

len(variables)

17

In [ ]:
for i in range(1, len(variables) + 1):
    for combination in itertools.combinations(variables, i):
        model = smf.ols(formula='DR ~ ' + ' + '.join(combination) + ' + 1', data=df).fit() # on rajoute '1' qui représente le constant dans le modèle
        models.append(model)
        y_hat = pd.DataFrame(model.predict(df[list(combination)]), columns = ['y'])
        y_pred.append(y_hat)

In [ ]:
print(len(models))
print(2**len(variables))

Le nombre total de modèles générés pour le BMA est de 131071, où 131071 représente le nombre de combinaisons de variables possible à partir de la base de données fournie.
p variables explicatives $\rightarrow 2^p$ modèles possibles

<div align="left"><span style="font-family:Lucida Caligraphy;font-size:25px;color:darkgreen"> <span style="color:green">      
Significativité des coefficients</span>

Cette étape permet de filtrer les modèles obtenus lors de la première étape en ne gardant que ceux ayant des coefficients statistiquement significatifs. Cela permet d'obtenir des modèles plus fiables et plus pertinents pour la prédiction du taux de défaut. La sortie affiche les résultats détaillés de chacun des modèles retenus.

In [ ]:
models_avec_coeffs_signif = []

for i in range (0,len(models)):    
    # Check if all p-values are less than 0.05
    all_p_values_less_than_005 = True
    for p_value in models[i].pvalues:
        if p_value >= 0.05:
            all_p_values_less_than_005 = False
            break
    if all_p_values_less_than_005:
        models_avec_coeffs_signif.append(i)

for elt in models_avec_coeffs_signif:
    print(models[elt].summary())

In [ ]:
len(models_avec_coeffs_signif)

La liste des modèles avec des coefficients significatifs contient 222 modèles qui ont tous leurs coefficients avec des p-values inférieures à 0.05, indiquant une forte probabilité que les coefficients soient non nuls.

<hr style="border-width:2px;border-color:green">
<center><h1>Tests des hypothèses économétriques</h1></center>
<hr style="border-width:2px;border-color:green">

Voici l'étape suivante de notre étude :

<div>
<p>
    Dans cette étape, nous allons vérifier si les modèles obtenus respectent les hypothèses de base de la régression linéaire multiple. 
</p>
<ul>
<li> Nous allons vérifier si les résidus ont une moyenne nulle : E(epsilon) = 0</li>
<li> Sont indépendants les uns des autres cov(epsilon_i, epsilon_j) = 0 pour i différent de j</li>
<li> Et sont homoscédastiques V(epsilon_i) = σ^2 : homoscédasticité</li>
<li> Epsilon est un vecteur gaussien</li>
<li>Nous allons également vérifier s'il n'y a pas de multicolinéarité entre les variables explicativesé</li>
</ul>
<p>
Pour ce faire, nous allons appliquer des tests statistiques et enlever les modèles qui ne respectent pas les hypothèses.
Nous allons également évaluer les métriques suivantes : RMSE, MSE, MAE, U-Theil, R2, R2 ajusté et intervalle de confiance. Le U-Theil est une métrique qui permet de mesurer la qualité de prédiction du modèle, mais il n'est pas souvent utilisé par les banques.
</p>
<p>
Nous prendrons ensuite le meilleur modèle qui optimise les métriques (RMSE, U-Theil, R2, etc.) pour nos prévisions.
</p>
</div>

<div align="left"><span style="font-family:Lucida Caligraphy;font-size:25px;color:darkgreen"> <span style="color:green">      
 Vérification de l'hypothèse 1 : $E(\epsilon) = 0$</span>


In [ ]:
for model in models : 
    # Calculate the residuals
    residuals = model.resid

    # Plot the residuals against the predicted values
    plt.scatter(model.predict(), residuals)
    plt.axhline(y=0, color='red')
    plt.xlabel('Predicted Values')
    plt.ylabel('Residuals')
    plt.show()

<div align="left"><span style="font-family:Lucida Caligraphy;font-size:25px;color:darkgreen"> <span style="color:green">      
 Vérification de l'hypothèse 2 : $cov(\epsilon_i, \epsilon_j) = 0$ for i $\neq$ j</span>


In [ ]:
for model in models : 
    # Calculate the residuals
    residuals = model.resid
    
    # Calculate the variance-covariance matrix of the residuals
    cov_matrix = np.cov(residuals)

    # Print the variance-covariance matrix
    print(cov_matrix)

<div align="left"><span style="font-family:Lucida Caligraphy;font-size:25px;color:darkgreen"> <span style="color:green">      
 Vérification de l'hypothèse 3 : $V(\epsilon_i) = \sigma^2$  (homoscédasticité)</span>



In [ ]:
for model in models : 
    plt.scatter(model.predict(), residuals ** 2)
    plt.xlabel('Predicted Values')
    plt.ylabel('Squared Residuals')
    plt.show()
    

 le test de white : 
 
 $H_0 :  V(\epsilon_i) = \sigma^2$
 
  $H_1 :  V(\epsilon_i) = \sigma_i^2$

In [ ]:
modeles_avec_resid_homosc = []

for i in range (0,len(models)):    
    # Calculate the residuals
    residuals = models[i].resid

    # perform White test
    white_test = het_white(residuals, models[i].model.exog)
    pvalue = white_test[1]
    
    if pvalue > 0.05:
        modeles_avec_resid_homosc.append(i)
    
len(modeles_avec_resid_homosc)

<div align="left"><span style="font-family:Lucida Caligraphy;font-size:25px;color:darkgreen"> <span style="color:green">      
Vérification de l'hypothèse 4 : $\epsilon$ est un vecteur gaussien</span>


In [ ]:
resid_norm = []

for i in range (0,len(models)):    
    # Calculate the residuals
    residuals = models[i].resid

    # Perform Shapiro-Wilk test
    stat, p = shapiro(residuals)
    # Interpret the results
    if p > 0.05:
        resid_norm.append(i)
    
len(resid_norm)

<div align="left"><span style="font-family:Lucida Caligraphy;font-size:25px;color:darkgreen"> <span style="color:green">      
Vérification de l'hypothèse 5 : pas de multi colinéarité</span>


Nous pouvons détecter la multicollinéarité en utilisant le facteur d'inflation de la variance (FIV). Sans entrer dans trop de détails, l'interprétation du FIV est la suivante : la racine carrée du FIV d'une variable donnée montre de combien la taille de l'erreur standard est plus grande, comparée à ce qu'elle serait si ce prédicteur était non corrélé avec les autres caractéristiques du modèle. Si aucune caractéristique n'est corrélée, alors toutes les valeurs de FIV seront de 1.
Pour traiter la multicollinéarité, nous devrions supprimer de manière itérative les caractéristiques avec des valeurs élevées de FIV. Une règle empirique pour la suppression pourrait être un FIV supérieur à 10 (5 est également courant).

In [ ]:
modeles_sns_multicol = []

for i in range (0,len(models)):    
    params = list(models[i].params.index.drop("Intercept"))
    combination = df[params]
    combination_cstt = add_constant(combination)
    vif = [variance_inflation_factor(combination_cstt.values, j) for j in range(combination_cstt.shape[1])]
    df_vif = pd.DataFrame({'vif': vif[1:]}, index=combination.columns)
    
    # check if any value in column is greater than 5
    if (df_vif['vif'] > 5).any():
        pass
    else : 
        modeles_sns_multicol.append(i)
            
len(modeles_sns_multicol)

<div align="left"><span style="font-family:Lucida Caligraphy;font-size:25px;color:darkgreen"> <span style="color:green">      
Sélection des modèles respectant les hypothèses</span>


Dans cette partie, nous allons effectuer une intersection entre les modèles qui satisfont les hypothèses de coefficients significatifs, de résidus homoscédastiques, de résidus gaussiens et de non-multicolinéarité. Cette intersection nous permettra d'identifier les modèles qui répondent à toutes les hypothèses en même temps, et donc de retenir le meilleur modèle pour la suite de notre analyse. Pour cela, nous utilisons la fonction set() qui permet de créer un ensemble, et nous effectuons une intersection entre les différents ensembles correspondants à chaque hypothèse. Ensuite, nous transformons cet ensemble en liste pour une meilleure lisibilité.

In [ ]:
intersection = set(models_avec_coeffs_signif).intersection(modeles_avec_resid_homosc, resid_norm, modeles_sns_multicol)
intersection = list(intersection)
len(intersection)

<div align="left"><span style="font-family:Lucida Caligraphy;font-size:25px;color:darkgreen"> <span style="color:green"> 
Au final, nous avons 62 modèles qui satisfont simultanément les critères de sélection suivants :</span></span></div>
<ul>
    <li><span style="font-size:16px">Les coefficients de régression sont significatifs (p-value inférieure à 0,05)</span></li>
    <li><span style="font-size:16px">Les résidus sont homoscédastiques</span></li>
    <li><span style="font-size:16px">Les résidus suivent une distribution normale</span></li>
    <li><span style="font-size:16px">Les variables indépendantes ne sont pas multicollinéaires</span></li>
</ul>
<span style="font-size:16px">Nous avons obtenu ces modèles en faisant une intersection entre les modèles avec des coefficients significatifs, les modèles avec des résidus homoscédastiques, les modèles avec des résidus suivant une distribution normale et les modèles sans multicollinéarité. Le nombre de modèles qui satisfont simultanément tous ces critères est de 62.</span>

In [ ]:
for elt in intersection:
    model = models[elt]
    print(elt, model.summary(), '\n \n \n \n')

<div align="left"><span style="font-family:Lucida Caligraphy;font-size:30px;color:darkgreen"> <span style="color:green">      
U de Theil</span>



L'étape suivante consiste à évaluer la qualité de notre modèle de prédiction en comparant ses prévisions avec une méthode naïve consistant à utiliser la valeur de la période précédente comme prédiction. Pour cela, on utilise l'U de Theil qui permet de mesurer la performance relative de notre modèle par rapport à cette méthode naïve. Si l'U de Theil est proche de 0 en valeur absolue, cela signifie que votre modèle est meilleur que l'estimateur naïf.*

$$ \text{U de Theil} = \sqrt{\frac{\Sigma_{t=1}^T (\frac{\hat{y_t}-y_t}{y_{t-1}})^2}{\Sigma_{t=1}^T (\frac{y_t-y_{t-1}}{y_{t-1}})^2}}$$


In [ ]:
default_rate2 = default_rate[8:] #pour qu'il est la même longueur que les y_pred
def u2(forecast,actual):
    num = ((forecast - actual)/ actual.shift(1))**2
    denom = ((actual - actual.shift(1))/ actual.shift(1))**2
    u = np.sqrt(np.sum(num)/np.sum(denom))
    return u

In [ ]:
modeles_retenus = []
for elt in intersection:
    if u2(y_pred[elt]["y"],default_rate2["DR"]) < 1:
        modeles_retenus.append(elt)
        print("Le modèle {} a un U de Theil de {}".format(elt, u2(y_pred[elt]["y"],default_rate2["DR"])))

<div align="left"><span style="font-family:Lucida Caligraphy;font-size:30px;color:darkgreen"> <span style="color:green">      
Modèles retenus </span>


In [ ]:
for elt in modeles_retenus:
    print(elt, models[elt].params, "\n\n")
    
#Les modeles retenus : 
# gr_UNR_lag3, DR_lag1 : modele 142
# gr_UNR_lag4, DR_lag1 : 146
# gr_RGDP_lag2, DR_lag1 : modele 86
# gr_RGDP_lag3, DR_lag1 : 97
# gr_RGDP_lag4, DR_lag1 : 107

<div align="left"><span style="font-family:Lucida Caligraphy;font-size:30px;color:darkgreen"> <span style="color:green">      
Sens économique</span>


In [ ]:
models[142].summary()

# Interprétation

Les résultats de notre modèle montrent que le taux de défaut retardé d'une période (DR_lag1) et le taux de chômage expliquent ensemble 95% de la variance du taux de défaut. De plus, ces variables sont toutes deux significatives, ce qui signifie qu'elles ont un impact significatif sur le taux de défaut. Enfin, le test de Fisher indique que le modèle est globalement significatif, ce qui renforce notre confiance dans sa capacité à expliquer le phénomène étudié.

Il est important de noter que la valeur de l'Omnibus est inférieure à 5, ce qui suggère que les résidus sont distribués normalement. Le test de Jarque-Bera (JB) est également non significatif, ce qui soutient l'hypothèse de normalité des résidus. Le coefficient de Durbin-Watson est proche de 2, ce qui suggère qu'il n'y a pas de corrélation sérielle dans les résidus.

Enfin, le modèle a un petit nombre d'observations (N=33), ce qui peut limiter sa généralisation à une population plus large. Il peut être intéressant d'explorer davantage les données pour voir si d'autres variables pourraient être incluses dans le modèle.

<hr style="border-width:2px;border-color:green">
<center><h1>Stationnarité de $ DR_t-\beta*DR_{t-1}$</h1></center>
<hr style="border-width:2px;border-color:green">

Nous prenons des valeurs de beta comprises entre 0 et 1 (exclus). Nous évaluons si la différence entre drt et beta drt-1 est stationnaire en traçant les p-value pour chaque valeur de beta sur trois graphiques séparés : un pour le test ADF, un pour le test KPSS et un pour le test PP.

Nous examinons chaque intervalle de beta pour lequel la série est stationnaire. Pour les trois tests, nous cherchons au moins deux intersections d'intervalles non vides. Nous pouvons alors conclure que la série est stationnaire si beta appartient à cet intervalle.

Enfin, nous estimons la valeur de drt - beta drt-1 et nous vérifions que beta appartient à l'intervalle précédemment identifié comme correspondant à une série stationnaire.

<div align="left"><span style="font-family:Lucida Caligraphy;font-size:30px;color:darkgreen"> <span style="color:green">      
Create an array of values for b</span>


Dans cette partie, nous allons créer un ensemble de séries chronologiques en appliquant une régression linéaire entre la série chronologique originale et sa version décalée d'une période, pour différentes valeurs de beta allant de 0.01 à 0.99 avec un pas de 0.01.

Nous allons stocker ces nouvelles séries chronologiques dans un DataFrame, en nommant chaque colonne selon la valeur de beta correspondante. Vous allons ensuite supprimer les valeurs manquantes (NaN) pour chaque colonne, car ces valeurs peuvent perturber les résultats des tests statistiques que vous allez effectuer par la suite.

In [ ]:

beta = np.arange(0.01, 1, 0.01)
df_vide = pd.DataFrame()
# Fit the linear regression model for each value of b
for b in beta:
    ts = default_rate - b*default_rate.shift(1)
    ts = ts.rename(columns = {"DR":"{}".format(b)})
    df_vide = pd.concat([df_vide, ts], axis = 1)

df_vide = df_vide.dropna()

In [ ]:
def stationnarite_pvalues(data, vars_list):
    res_df = pd.DataFrame(columns=['BETA', 'P-VALUE FOR ADF TEST',
                                    'P-VALUE FOR PP TEST',
                                   'P-VALUE FOR KPSS TEST'])
    loop = 1
    for x in vars_list:        
        adf_result = adfuller(data[x])
        pp_result = PhillipsPerron(data[x])
        kpss_result = kpss(data[x])         
        res_df = res_df.append({'BETA': x.upper(),
                                'P-VALUE FOR ADF TEST': adf_result[1],
                                'P-VALUE FOR PP TEST': pp_result.pvalue,
                                'P-VALUE FOR KPSS TEST': kpss_result[1],
                                '_i': loop}, ignore_index=True)
        loop += 1
    res_df = res_df.sort_values(by=['_i'])
    del res_df['_i']
    return res_df

In [ ]:
p_values = stationnarite_pvalues(df_vide, df_vide.columns)
p_values.head()

In [ ]:
for column in ["P-VALUE FOR ADF TEST", "P-VALUE FOR PP TEST", "P-VALUE FOR KPSS TEST"]:
    
    p_values = p_values.sort_values(by=column)

    # Plot the data
    sns.lineplot(data=p_values, x="BETA", y=column)

    # Draw a horizontal line at y=0.05
    plt.axhline(y=0.05, color='red', linestyle='--')

    p_values["BETA"] = p_values["BETA"].astype(float)

    # Get the x value where the horizontal line intersects the plot
    x_intercept = np.interp(0.05, p_values[column], p_values["BETA"])

    # Draw a vertical line from the horizontal line to the x axis
    plt.axvline(x=x_intercept, color='green', linestyle='--')

    # Show the plot
    plt.show()
    print(x_intercept)

# Commentaire 

En utilisant le test KPSS, on peut conclure que la série "drt - beta drt-1" est stationnaire lorsque "beta" appartient à l'intervalle [0.8323:1[. De même, en utilisant le test PP, on peut conclure que la série est stationnaire lorsque "beta" appartient à l'intervalle [0.643, 1[. Ainsi, on peut affirmer que la série "drt - beta drt-1" est stationnaire lorsque "beta" appartient à l'intervalle [0.8323:1[.

<div align="left"><span style="font-family:Lucida Caligraphy;font-size:30px;color:darkgreen"> <span style="color:green">      
Estimatimation du taux de défaut </span>


Dans cette partie on utilise la méthode d'estimation des moindres carrés ordinaires (OLS) pour estimer un modèle de régression linéaire qui lie le taux de défaut de la période actuelle à celui de la période précédente. Ensuite, en se basant sur les résultats des tests ADF, KPSS et PP effectués précédemment, on vérifie si le coefficient de régression beta obtenu appartient à l'intervalle [0.8323:1[ qui permet d'affirmer que la série temporelle est stationnaire. Le fait que la valeur de beta soit égale à 0.9250, qui appartient bien à l'intervalle déterminé, permet donc de conclure que la série est stationnaire.

In [ ]:
#on estime drt - beta drt-1  et on verifie que beta appartienne à cet intervalle.
model = smf.ols(formula='default_rate ~ default_rate.shift(1)' , data=default_rate).fit()
model.summary()

# 0.9250  appartient bien à [0.8323:1[

<hr style="border-width:2px;border-color:green">
<center><h1>Application du modèle sur les base sur des différents scénarios macroéconomiques</h1></center>
<hr style="border-width:2px;border-color:green">


Dans cette partie, nous appliquons notre modèle à deux jeux de données: var_macro_adverse et var_macro_baseline. Pour chaque jeu de données, nous générons des scénarios de taux de croissance et réorganisons les données en renommant les colonnes et en ajustant les valeurs numériques.

In [ ]:
scenarios_taux = taux_croissance(var_macro_adverse)
var_macro_adverse = pd.concat([scenarios_taux["gr_UNR"], var_macro_adverse.drop(columns = ["UNR"])], axis = 1).dropna()
var_macro_adverse = var_macro_adverse.rename(columns= {"RGDP":"gr_RGDP", "HICP":"gr_HICP", "RREP":"gr_RREP", "IRLT":"gr_IRLT"})
var_macro_adverse[["gr_RGDP" ,"gr_HICP" ,"gr_RREP"]] = var_macro_adverse[["gr_RGDP" ,"gr_HICP" ,"gr_RREP"]] / 100
var_macro_adverse["gr_IRLT"] = var_macro_adverse["gr_IRLT"] / 10

scenarios_taux = taux_croissance(var_macro_baseline)
var_macro_baseline = pd.concat([scenarios_taux["gr_UNR"], var_macro_baseline.drop(columns = ["UNR"])], axis = 1).dropna()
var_macro_baseline = var_macro_baseline.rename(columns= {"RGDP":"gr_RGDP", "HICP":"gr_HICP", "RREP":"gr_RREP", "IRLT":"gr_IRLT"})
var_macro_baseline[["gr_RGDP" ,"gr_HICP" ,"gr_RREP"]] = var_macro_baseline[["gr_RGDP" ,"gr_HICP" ,"gr_RREP"]] / 100
var_macro_baseline["gr_IRLT"] = var_macro_baseline["gr_IRLT"] / 10

<div align="left"><span style="font-family:Lucida Caligraphy;font-size:30px;color:darkgreen"> <span style="color:green">      
Estimatimation de nos modèles rétenus </span>


In [ ]:
model_142 = smf.ols(formula='DR ~ gr_UNR_lag3 + DR_lag1  + 1', data=df).fit() #pour ne pas avoir à run le BMA
model_142.params

In [ ]:
model_86 = smf.ols(formula='DR ~ gr_RGDP_lag2 + DR_lag1  + 1', data=df).fit() #pour ne pas avoir à run le BMA
model_86.params

<div align="left"><span style="font-family:Lucida Caligraphy;font-size:30px;color:darkgreen"> <span style="color:green">      
scénarios macroéconomiques. </span>



Les bases de données fournies pour nos tests comprennent des données trimestrielles pour les années 2018, 2019 et 2020. On constate que les valeurs pour chaque trimestre sont les mêmes dans les bases de données pour les deux scénarios, à savoir "baseline" et "adverse". Ces bases de données sont utilisées pour tester la capacité de notre modèle à prédire le taux de défaut.

Nous allons utiliser ces deux bases de données pour produire des graphiques permettant de visualiser les prévisions de notre modèle pour les taux de défaut dans les scénarios "baseline" et "adverse". Les graphiques nous aideront à évaluer la performance de notre modèle et à déterminer s'il est capable de prédire de manière fiable les taux de défaut dans différents scénarios macroéconomiques.

<table>
  <thead>
    <tr>
      <th rowspan="2">Date</th>
      <th colspan="5">Baseline</th>
      <th colspan="5">Adverse</th>
    </tr>
    <tr>
      <th>RGDP</th>
      <th>HICP</th>
      <th>RREP</th>
      <th>IRLT</th>
      <th>UNR</th>
      <th>RGDP</th>
      <th>HICP</th>
      <th>RREP</th>
      <th>IRLT</th>
      <th>UNR</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>2018-01-31</td>
      <td>1.4</td>
      <td>0.9</td>
      <td>1.5</td>
      <td>2.1</td>
      <td>11.1</td>
      <td>-0.6</td>
      <td>0.8</td>
      <td>-7.3</td>
      <td>3.3</td>
      <td>11.3</td>
    </tr>
    <tr>
      <td>2018-04-30</td>
      <td>1.4</td>
      <td>0.9</td>
      <td>1.5</td>
      <td>2.1</td>
      <td>11.1</td>
      <td>-0.6</td>
      <td>0.8</td>
      <td>-7.3</td>
      <td>3.3</td>
      <td>11.3</td>
    </tr>
    <tr>
      <td>2018-07-31</td>
      <td>1.4</td>
      <td>0.9</td>
      <td>1.5</td>
      <td>2.1</td>
      <td>11.1</td>
      <td>-0.6</td>
      <td>0.8</td>
      <td>-7.3</td>
      <td>3.3</td>
      <td>11.3</td>
    </tr>
    <tr>
      <td>2018-10-31</td>
      <td>1.4</td>
      <td>0.9</td>
      <td>1.5</td>
      <td>2.1</td>
      <td>11.1</td>
      <td>-0.6</td>
      <td>0.8</td>
      <td>-7.3</td>
      <td>3.3</td>
      <td>11.3</td>
    </tr>
    <tr>
      <td>2019-01-31</td>
      <td>1.3</td>
      <td>1.5</td>
      <td>2.2</td>
      <td>2.5</td>
      <td>10.8</td>
      <td>-1.5</td>
      <td>0.8</td>
      <td>-4.9</td



In [ ]:
graphiques(var_macro_baseline)


In [ ]:
graphiques(var_macro_adverse)

<hr style="border-width:2px;border-color:green">
<center><h1>Modèle 1</h1></center>
<hr style="border-width:2px;border-color:green">


On applique le même traitement fait sur la base var_macro 

In [ ]:
def prediction_modele_142(scenario) : 
    sc = create_lags_diffs(scenario, num_lags = 3)
    X1 = sc["gr_UNR_lag3"] 
    X = pd.concat([X1, df[-1:]["DR_lag1"], df[-1:]["DR"]], axis = 1) #on rajoute la 2ème variable explicative
    dates =  X.index.date.astype(str).tolist()
    
    for i in range (len(dates)-1) :
        X_1 = X.loc[dates[i]] # au début on retient juste 2019-10-31 pour calculer la date d'apres, etc.
        sc_predictions = model_142.predict(X_1)
        sc_predictions = pd.DataFrame(sc_predictions, columns = ["DR_pred"])
        X.loc[dates[i+1]]['DR'] = sc_predictions["DR_pred"]
        X["DR_lag1"] = X["DR"].shift(1)
        
    serie_predite = X.loc[X.index > '2019-10-31']["DR"]
    serie_predite_et_val_precente = X.loc[X.index > '2019-07-31']["DR"] #pour qu'il y ait pas de trou dans le plot dû aux dates
    return serie_predite, serie_predite_et_val_precente

def plot_modele_142(scenario) : 
    plt.plot(default_rate, label='Série originale')
    plt.plot(prediction_modele_142(scenario)[1], label='Série prédite')
    plt.legend()
    plt.show()

In [ ]:
prediction_modele_142(var_macro_adverse)

# 2020-01-31    0.030860
# 2020-04-30    0.029303
# 2020-07-31    0.031741
# 2020-10-31    0.030438

In [ ]:
plot_modele_142(var_macro_baseline)


In [ ]:
plot_modele_142(var_macro_adverse)

<hr style="border-width:2px;border-color:green">
<center><h1>Modèle 2</h1></center>
<hr style="border-width:2px;border-color:green">


In [ ]:

def prediction_modele_86(scenario) : 
    sc = create_lags_diffs(scenario, num_lags = 3)
    X1 = sc["gr_RGDP_lag2"] 
    X = pd.concat([X1, df[-1:]["DR_lag1"], df[-1:]["DR"]], axis = 1) #on rajoute la 2ème variable explicative
    dates =  X.index.date.astype(str).tolist()
    
    for i in range (len(dates)-1) :
        X_1 = X.loc[dates[i]] # au début on retient juste 2019-10-31 pour calculer la date d'apres, etc.
        sc_predictions = model_86.predict(X_1)
        sc_predictions = pd.DataFrame(sc_predictions, columns = ["DR_pred"])
        X.loc[dates[i+1]]['DR'] = sc_predictions["DR_pred"]
        X["DR_lag1"] = X["DR"].shift(1)
        
    serie_predite = X.loc[X.index > '2019-10-31']["DR"]
    serie_predite_et_val_precente = X.loc[X.index > '2019-07-31']["DR"] #pour qu'il y ait pas de trou dans le plot dû aux dates
    return serie_predite, serie_predite_et_val_precente

In [ ]:
def plot_modele_86(scenario) : 
    plt.plot(default_rate, label='Série originale')
    plt.plot(prediction_modele_86(scenario)[1], label='Série prédite')
    plt.legend()
    plt.show()

In [ ]:
plot_modele_86(var_macro_adverse)

In [ ]:
plot_modele_86(var_macro_baseline)

<hr style="border-width:2px;border-color:green">
<center><h1>Modèle 3 : Stacking</h1></center>
<hr style="border-width:2px;border-color:green">


In [ ]:
y_hat_142 = model_142.predict(df)
y_hat_86 = model_86.predict(df)

In [ ]:
#et si on les stackait ?

# Stack the predictions into a matrix
preds_stacked = np.column_stack((y_hat_142, y_hat_86))

# Fit a new OLS model using the stacked predictions
dr = default_rate[7:]
model_stacked = sm.OLS(dr, preds_stacked).fit()

# Print the summary statistics for the new model
print(model_stacked.summary())

In [ ]:
model_stacked.summary()

In [ ]:
def pred_modele_stacking(scenario) :


    # Get the predictions from each of the original models for the new data
    preds1_new = prediction_modele_86(scenario)[1]
    preds2_new = prediction_modele_142(scenario)[1]

    # Stack the new predictions into a matrix
    preds_stacked_new = np.column_stack((preds1_new, preds2_new))

    # Make predictions using the stacked model
    y_pred_stacked = model_stacked.predict(preds_stacked_new)
    y_pred_stacked = pd.DataFrame(y_pred_stacked, columns = ["DR"])
    index_values = ['2019-10-31','2020-01-31','2020-04-30','2020-07-31','2020-10-31']
    y_pred_stacked['index'] = index_values
    y_pred_stacked = y_pred_stacked.set_index('index')
    y_pred_stacked.index = pd.to_datetime(y_pred_stacked.index)
    return y_pred_stacked

def plot_modele_stacking(scenario) : 
    plt.plot(default_rate, label='Série originale')
    plt.plot(pred_modele_stacking(scenario), label='Série prédite')
    plt.legend()
    plt.show()

In [ ]:
plot_modele_stacking(var_macro_baseline)

In [ ]:
plot_modele_stacking(var_macro_adverse)